In [66]:
# Divide up into cars and notcars
car_images = glob.glob('./vehicles/*/*.png')
notcar_images = glob.glob('./non-vehicles/*/*.png')
cars = []
notcars = []
for image in car_images:
    cars.append(image)
        
for image in notcar_images:
    # if 'image' in image or 'extra' in image:
    # if 'extra' in image:
    #     notcars.append(image)
    # else:
    notcars.append(image)

print(len(cars), len(notcars))

8792

8968

In [67]:
import random
fig = plt.figure(figsize=(6,6))
for i in range(1,13,2):
    img = mpimg.imread(cars[random.randint(1,1000)])
    plt.subplot(6,2,i)
    plt.imshow(cv2.resize(img, (128, 128)))
    # plt.title('Cars images')
    img = mpimg.imread(notcars[random.randint(1,1000)])
    plt.subplot(6,2,i+1)
    plt.imshow(cv2.resize(img, (128, 128)))
    # plt.title('Not-car image')
fig.tight_layout()
fig.savefig('./output_images/cars_notcars.jpg')
# plt.show()

In [68]:
from skimage.feature import hog
import cv2

def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)


def get_hog_features(img, orient, pix_per_cell, cell_per_block,
                     vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient,
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  transform_sqrt=False,
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:
        features = hog(img, orientations=orient,
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       transform_sqrt=False,
                       visualise=vis, feature_vector=feature_vec)
        return features


def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:, :, 0], size).ravel()
    color2 = cv2.resize(img[:, :, 1], size).ravel()
    color3 = cv2.resize(img[:, :, 2], size).ravel()
    return np.hstack((color1, color2, color3))


def color_hist(img, nbins=32):  # bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:, :, 0], bins=nbins)
    channel2_hist = np.histogram(img[:, :, 1], bins=nbins)
    channel3_hist = np.histogram(img[:, :, 2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features


def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9,
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel],
                                        orient, pix_per_cell, cell_per_block,
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient,
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features

In [69]:
img = mpimg.imread(cars[random.randint(1,1000)])
f, v = get_hog_features(cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)[:,:,0], 9, 8, 2,
                     vis=True, feature_vec=True)
fig = plt.figure(figsize=(4,4))

plt.subplot(121)
plt.imshow(cv2.resize(img, (128,128)))
plt.title('training image')
plt.subplot(122)
plt.imshow(cv2.resize(v, (128,128)))
plt.title('hog features')
fig.savefig('./output_images/hog_training.jpg')

In [70]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
import os
import pickle
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
from sklearn.model_selection import train_test_split

colorspace = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32)
hist_bins = 32
spatial_feat = True

t=time.time()
car_features = extract_features(cars, color_space=colorspace, orient=orient,
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                        hog_channel=hog_channel, spatial_size=spatial_size, hist_bins=hist_bins,
                                spatial_feat=spatial_feat,
                                )
notcar_features = extract_features(notcars, color_space=colorspace, orient=orient,
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                        hog_channel=hog_channel, spatial_size=spatial_size, hist_bins=hist_bins,
                                   spatial_feat=spatial_feat,
                                   )
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')
dist_pickle = {}
dist_pickle["svc"] = svc
dist_pickle["scaler"] = X_scaler
dist_pickle["orient"] = orient
dist_pickle["pix_per_cell"] = pix_per_cell
dist_pickle["cell_per_block"] = cell_per_block
dist_pickle["spatial_size"] = spatial_size
dist_pickle["hist_bins"] = hist_bins
dist_pickle["spatial_feat"] = spatial_feat


pickle.dump(dist_pickle, open("svc_pickle.p", "wb" ))
print('written test data to pickle file')

92.57

Seconds to extract HOG features...

Using:

9

orientations

...

0.02006

Seconds to predict

10

labels with SVC

written test data to pickle file

In [71]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap  # Iterate through list of bboxes


def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap


def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1] + 1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        color = [0, 0, 0]
        color[car_number % 3] = 255
        cv2.rectangle(img, bbox[0], bbox[1], color, 6)
    # Return the image
    return img

In [72]:
class Frame():
    def __init__(self):
        self._current_heat = None
        self.n_heats = []
        self._current_labels = None
        self.n_labels = []
        self.n = 6

    @property
    def current_heat(self):
        return self._current_heat

    @current_heat.setter
    def current_heat(self, heat):
        self.n_heats.append(heat)
        if len(self.n_heats) > self.n:
            self.n_heats.pop(0)
        self._current_heat = heat

    @property
    def current_labels(self):
        return self._current_labels

    @current_labels.setter
    def current_labels(self, labels):
        self.n_labels.append(labels[0])
        if len(self.n_labels) > self.n:
            self.n_labels.pop(0)
        self._current_labels = labels

    @property
    def avg_heat(self):
        return np.mean(self.n_heats, axis=0, dtype=int)

    @property
    def avg_labels(self):
        return np.mean(self.n_labels, axis=0, dtype=int)

In [73]:
# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins,
              spatial_feat=True, hist_feat=True, hog_feat=True):
    draw_img = np.copy(img)
    # img = img.astype(np.float32) / 255

    img_tosearch = img[ystart:ystop, :, :]
    # ctrans_tosearch = project.convert_color(img_tosearch, conv='RGB2YCrCb')
    # ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2HLS)
    ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2YCrCb)
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1] / scale), np.int(imshape[0] / scale)))

    ch1 = ctrans_tosearch[:, :, 0]
    ch2 = ctrans_tosearch[:, :, 1]
    ch3 = ctrans_tosearch[:, :, 2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1
    nfeat_per_block = orient * cell_per_block ** 2

    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step

    # Compute individual channel HOG features for the entire image
    hog1 = project.get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = project.get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = project.get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)

    on_windows = []
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb * cells_per_step
            xpos = xb * cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos + nblocks_per_window, xpos:xpos + nblocks_per_window].ravel()
            hog_feat2 = hog2[ypos:ypos + nblocks_per_window, xpos:xpos + nblocks_per_window].ravel()
            hog_feat3 = hog3[ypos:ypos + nblocks_per_window, xpos:xpos + nblocks_per_window].ravel()
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos * pix_per_cell
            ytop = ypos * pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop + window, xleft:xleft + window], (64, 64))

            # Get color features
            spatial_features = project.bin_spatial(subimg, size=spatial_size)
            hist_features = project.color_hist(subimg, nbins=hist_bins)
            features = []
            if spatial_feat == True:
                features.append(spatial_features)
            if hist_feat == True:
                features.append(hist_features)
            if hog_feat == True:
                features.append(hog_features)

            # Scale features and make a prediction
            test_features = X_scaler.transform(
                np.hstack(features).reshape(1, -1))
            # test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))
            # test_prediction = svc.predict(test_features)
            test_prediction = svc.predict(test_features)

            if test_prediction == 1:
                xbox_left = np.int(xleft * scale)
                ytop_draw = np.int(ytop * scale)
                win_draw = np.int(window * scale)
                # cv2.rectangle(draw_img, (xbox_left, ytop_draw + ystart),
                #               (xbox_left + win_draw, ytop_draw + win_draw + ystart), (0, 0, 255), 6)
                on_windows.append(((xbox_left, ytop_draw + ystart),
                              (xbox_left + win_draw, ytop_draw + win_draw + ystart)))
    return on_windows

In [74]:
def find_cars_pipeline(img):
    ystart = 400
    ystop = 656
    hot_windows = []
    for scale in scales:
        hot_windows += find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size,
                        hist_bins, spatial_feat=spatial_feat)
    heat = np.zeros_like(img[:, :, 0]).astype(np.float32)
    heat = add_heat(heat, hot_windows)
    heat = apply_threshold(heat, 2)
    frame.current_heat = heat
    heatmap = np.clip(heat, 0, 255)
    labels = label(heatmap)
    out_img = draw_labeled_bboxes(np.copy(img), label(apply_threshold(frame.avg_heat, 2)))
    # out_img = draw_labeled_bboxes(np.copy(img), labels)
    return out_img

In [75]:
import matplotlib.pyplot as plt

# load parameters from pickle file
dist_pickle = pickle.load( open("svc_pickle.p", "rb" ) )
svc = dist_pickle["svc"]
X_scaler = dist_pickle["scaler"]
orient = dist_pickle["orient"]
pix_per_cell = dist_pickle["pix_per_cell"]
cell_per_block = dist_pickle["cell_per_block"]
spatial_size = dist_pickle["spatial_size"]
hist_bins = dist_pickle["hist_bins"]
spatial_feat = dist_pickle["spatial_feat"]

ystart = 400
ystop = 656
scales = [1, 1.5, 2, 2.5, 3]
images = glob.glob('./test_images/clip_25*.jpg')
frame = Frame()
i=0

fig = plt.figure(figsize=(8,24))
for image_name in images:
    i+=2
    img = mpimg.imread(image_name)
    # img = cv2.imread(image_name)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hot_windows = []
    for scale in scales:
        hot_windows += find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size,
                        hist_bins, spatial_feat=spatial_feat)
    heat = np.zeros_like(img[:, :, 0]).astype(np.float32)
    heat = add_heat(heat, hot_windows)
    frame.current_heat = heat
    heat = apply_threshold(heat, 2)
    heatmap = np.clip(heat, 0, 255)
    # cv2.imwrite('./output_images/avg_heat_{0}'.format(os.path.basename(image_name)), heat * 25)
    labels = label(heatmap)
    frame.current_labels = labels
    out_img_avg_heat = draw_labeled_bboxes(np.copy(img), labels)
    # cv2.imwrite('./output_images/avg_heat_labels_{0}'.format(os.path.basename(image_name)), cv2.cvtColor(out_img_avg_heat, cv2.COLOR_RGB2BGR))
    plt.subplot(6,2,i-1)
    plt.imshow(out_img_avg_heat)
    plt.title('Car Positions')
    plt.subplot(6,2,i)
    plt.imshow(heat, cmap='hot')
    plt.title('Heat Map')
fig.tight_layout()
fig.savefig('./output_images/all_heatmap')
# plt.show()

out_img_avg_heat = draw_labeled_bboxes(np.copy(img), label(apply_threshold(frame.avg_heat, 2)))
fig = plt.figure()
plt.subplot(121)
plt.imshow(out_img_avg_heat)
plt.title('Car Positions')
plt.subplot(122)
plt.imshow(frame.avg_heat, cmap='hot')
plt.title('Heat Map')
fig.tight_layout()
fig.savefig('./output_images/averaged_heatmap')
# plt.show()

In [11]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import matplotlib.image as mpimg
import cv2
import glob
import os
import pickle
from scipy.ndimage.measurements import label

# 
# left = Line()
# right = Line()
# left_fitx = None
# right_fitx = None
dist_pickle = pickle.load( open("svc_pickle.p", "rb" ) )
svc = dist_pickle["svc"]
X_scaler = dist_pickle["scaler"]
orient = dist_pickle["orient"]
pix_per_cell = dist_pickle["pix_per_cell"]
cell_per_block = dist_pickle["cell_per_block"]
spatial_size = dist_pickle["spatial_size"]
hist_bins = dist_pickle["hist_bins"]
spatial_feat = dist_pickle["spatial_feat"]

frame = Frame()
scales = [1, 1.5, 2, 2.5, 3]
cars_output = 'output_video/project_4.mp4'
clip1 = VideoFileClip("project_video.mp4")
cars_clip = clip1.fl_image(find_cars_pipeline) #NOTE: this function expects color images!!
%time cars_clip.write_videofile(cars_output, audio=False)

[MoviePy] >>>> Building video output_video/project_3.mp4


[MoviePy] Writing video output_video/project_3.mp4


  0%|          | 0/1261 [00:00<?, ?it/s]

  0%|          | 1/1261 [00:01<34:01,  1.62s/it]

  0%|          | 2/1261 [00:03<33:47,  1.61s/it]

...

100%|█████████▉| 1259/1261 [33:36<00:03,  1.55s/it]

100%|█████████▉| 1260/1261 [33:38<00:01,  1.55s/it]

[MoviePy] Done.


[MoviePy] >>>> Video ready: output_video/project_3.mp4 



Wall time: 33min 38s